## Load SQuAD data

In [13]:
import os
import numpy as np
import pandas as pd
from transformers.agents import agent_types
from tqdm.notebook import tqdm
import logging
from IPython.display import Markdown
from semscore import EmbeddingModelWrapper
from statistics import mean
from agent import get_agent
from dotenv import load_dotenv
from prompts import SUCCINCT_SQUAD_REACT_CODE_SYSTEM_PROMPT, FOCUSED_SQUAD_REACT_CODE_SYSTEM_PROMPT

load_dotenv()  # Load OPENAI_API_KEY from .env (not included in repo)

def display_text_df(df):
    display(df.style.set_properties(**{'white-space': 'pre-wrap'}).set_table_styles(
        [{'selector': 'th', 'props': [('text-align', 'left')]},
         {'selector': 'td', 'props': [('text-align', 'left')]}
        ]
    ).hide())


In [2]:
from data import get_data
data = get_data(download=False)


In [3]:
data.df.shape

(87599, 4)

In [4]:
display_text_df(data.df.head(3))


Title,Context,Question,Answer
University_of_Notre_Dame,"Architecturally, the school has a Catholic character. Atop the Main Building's gold dome is a golden statue of the Virgin Mary. Immediately in front of the Main Building and facing it, is a copper statue of Christ with arms upraised with the legend ""Venite Ad Me Omnes"". Next to the Main Building is the Basilica of the Sacred Heart. Immediately behind the basilica is the Grotto, a Marian place of prayer and reflection. It is a replica of the grotto at Lourdes, France where the Virgin Mary reputedly appeared to Saint Bernadette Soubirous in 1858. At the end of the main drive (and in a direct line that connects through 3 statues and the Gold Dome), is a simple, modern stone statue of Mary.",To whom did the Virgin Mary allegedly appear in 1858 in Lourdes France?,Saint Bernadette Soubirous
University_of_Notre_Dame,"Architecturally, the school has a Catholic character. Atop the Main Building's gold dome is a golden statue of the Virgin Mary. Immediately in front of the Main Building and facing it, is a copper statue of Christ with arms upraised with the legend ""Venite Ad Me Omnes"". Next to the Main Building is the Basilica of the Sacred Heart. Immediately behind the basilica is the Grotto, a Marian place of prayer and reflection. It is a replica of the grotto at Lourdes, France where the Virgin Mary reputedly appeared to Saint Bernadette Soubirous in 1858. At the end of the main drive (and in a direct line that connects through 3 statues and the Gold Dome), is a simple, modern stone statue of Mary.",What is in front of the Notre Dame Main Building?,a copper statue of Christ
University_of_Notre_Dame,"Architecturally, the school has a Catholic character. Atop the Main Building's gold dome is a golden statue of the Virgin Mary. Immediately in front of the Main Building and facing it, is a copper statue of Christ with arms upraised with the legend ""Venite Ad Me Omnes"". Next to the Main Building is the Basilica of the Sacred Heart. Immediately behind the basilica is the Grotto, a Marian place of prayer and reflection. It is a replica of the grotto at Lourdes, France where the Virgin Mary reputedly appeared to Saint Bernadette Soubirous in 1858. At the end of the main drive (and in a direct line that connects through 3 statues and the Gold Dome), is a simple, modern stone statue of Mary.",The Basilica of the Sacred heart at Notre Dame is beside to which structure?,the Main Building


In [5]:
np.random.seed(42)
# Select 10 random rows from data.df
dfSample = data.df.sample(n=10)
display_text_df(dfSample)

Title,Context,Question,Answer
Institute_of_technology,"The world's first institution of technology or technical university with tertiary technical education is the Banská Akadémia in Banská Štiavnica, Slovakia, founded in 1735, Academy since December 13, 1762 established by queen Maria Theresa in order to train specialists of silver and gold mining and metallurgy in neighbourhood. Teaching started in 1764. Later the department of Mathematics, Mechanics and Hydraulics and department of Forestry were settled. University buildings are still at their place today and are used for teaching. University has launched the first book of electrotechnics in the world.",What year was the Banská Akadémia founded?,1735
Film_speed,"The standard specifies how speed ratings should be reported by the camera. If the noise-based speed (40:1) is higher than the saturation-based speed, the noise-based speed should be reported, rounded downwards to a standard value (e.g. 200, 250, 320, or 400). The rationale is that exposure according to the lower saturation-based speed would not result in a visibly better image. In addition, an exposure latitude can be specified, ranging from the saturation-based speed to the 10:1 noise-based speed. If the noise-based speed (40:1) is lower than the saturation-based speed, or undefined because of high noise, the saturation-based speed is specified, rounded upwards to a standard value, because using the noise-based speed would lead to overexposed images. The camera may also report the SOS-based speed (explicitly as being an SOS speed), rounded to the nearest standard speed rating.",What is another speed that can also be reported by the camera?,SOS-based speed
Sumer,"The most impressive and famous of Sumerian buildings are the ziggurats, large layered platforms which supported temples. Sumerian cylinder seals also depict houses built from reeds not unlike those built by the Marsh Arabs of Southern Iraq until as recently as 400 CE. The Sumerians also developed the arch, which enabled them to develop a strong type of dome. They built this by constructing and linking several arches. Sumerian temples and palaces made use of more advanced materials and techniques,[citation needed] such as buttresses, recesses, half columns, and clay nails.",Where were the use of advanced materials and techniques on display in Sumer?,Sumerian temples and palaces
"Ann_Arbor,_Michigan","Ann Arbor has a council-manager form of government. The City Council has 11 voting members: the mayor and 10 city council members. The mayor and city council members serve two-year terms: the mayor is elected every even-numbered year, while half of the city council members are up for election annually (five in even-numbered and five in odd-numbered years). Two council members are elected from each of the city's five wards. The mayor is elected citywide. The mayor is the presiding officer of the City Council and has the power to appoint all Council committee members as well as board and commission members, with the approval of the City Council. The current mayor of Ann Arbor is Christopher Taylor, a Democrat who was elected as mayor in 2014. Day-to-day city operations are managed by a city administrator chosen by the city council.",Who is elected every even numbered year?,mayor
John_von_Neumann,"Shortly before his death, when he was already quite ill, von Neumann headed the United States government's top secret ICBM committee, and it would sometimes meet in his home. Its purpose was to decide on the feasibility of building an ICBM large enough to carry a thermonuclear weapon. Von Neumann had long argued that while the technical obstacles were sizable, they could be overcome in time. The SM-65 Atlas passed its first fully functional test in 1959, two years after his death. The feasibility of an ICBM owed as much to improved, smaller warheads as it did to developments in rocketry, and his understanding of the former made his advice invaluable.",What was the purpose

In [6]:
# use local gpt to synthesize questions with context

synth_system_prompt = """
You are an expert at clarifying what questions are really asking for.

You will be given a question, a title and context.
Your task is come up with a new version of the question that resolves ambiguities by adding only and exactly the necessary details from the title and context in a way that clarifies the question without changing the meaning or intent of the question.

For example: 
Question: Who does M fight with?
Title: Spectre_(2015_film)
Context: Bond and Swann return to London where they meet M, Bill Tanner, Q, and Moneypenny; they intend to arrest C and stop Nine Eyes from going online. Swann leaves Bond, telling him she cannot be part of a life involving espionage, and is subsequently kidnapped. On the way, the group is ambushed and Bond is kidnapped, but the rest still proceed with the plan. After Q succeeds in preventing the Nine Eyes from going online, a brief struggle between M and C ends with the latter falling to his death. Meanwhile, Bond is taken to the old MI6 building, which is scheduled for demolition, and frees himself. Moving throughout the ruined labyrinth, he encounters a disfigured Blofeld, who tells him that he has three minutes to escape the building before explosives are detonated or die trying to save Swann. Bond finds Swann and the two escape by boat as the building collapses. Bond shoots down Blofeld's helicopter, which crashes onto Westminster Bridge. As Blofeld crawls away from the wreckage, Bond confronts him but ultimately leaves him to be arrested by M. Bond leaves the bridge with Swann.
Response: Who does M struggle with during the events of Spectre (2015)?
"""

synth_user_prompt = """
Question: {question}
Title: {title}
Context: {context}
"""


In [7]:
# Example: reuse your existing OpenAI setup
from openai import OpenAI

# Point to the local server
# client = OpenAI(base_url="http://localhost:1234/v1", api_key="lm-studio")
client = OpenAI()

synth_answers = []
for title, context, question, answer in tqdm(dfSample.values):
    completion = client.chat.completions.create(
    # model="lmstudio-community/Meta-Llama-3.1-8B-Instruct-GGUF",
    model="gpt-4o-mini-2024-07-18",
    messages=[
        {"role": "system", "content": synth_system_prompt},
        {"role": "user", "content": synth_user_prompt.format(question=question, title=title, context=context)}
    ],
    temperature=0.7,
    )
    synth_answers.append(completion.choices[0].message.content)

dfSample["Synthesized Question"] = synth_answers
display_text_df(dfSample)

  0%|          | 0/10 [00:00<?, ?it/s]

Title,Context,Question,Answer,Synthesized Question
Institute_of_technology,"The world's first institution of technology or technical university with tertiary technical education is the Banská Akadémia in Banská Štiavnica, Slovakia, founded in 1735, Academy since December 13, 1762 established by queen Maria Theresa in order to train specialists of silver and gold mining and metallurgy in neighbourhood. Teaching started in 1764. Later the department of Mathematics, Mechanics and Hydraulics and department of Forestry were settled. University buildings are still at their place today and are used for teaching. University has launched the first book of electrotechnics in the world.",What year was the Banská Akadémia founded?,1735,"What year was the Banská Akadémia, the world's first institution of technology, founded in Banská Štiavnica, Slovakia?"
Film_speed,"The standard specifies how speed ratings should be reported by the camera. If the noise-based speed (40:1) is higher than the saturation-based speed, the noise-based speed should be reported, rounded downwards to a standard value (e.g. 200, 250, 320, or 400). The rationale is that exposure according to the lower saturation-based speed would not result in a visibly better image. In addition, an exposure latitude can be specified, ranging from the saturation-based speed to the 10:1 noise-based speed. If the noise-based speed (40:1) is lower than the saturation-based speed, or undefined because of high noise, the saturation-based speed is specified, rounded upwards to a standard value, because using the noise-based speed would lead to overexposed images. The camera may also report the SOS-based speed (explicitly as being an SOS speed), rounded to the nearest standard speed rating.",What is another speed that can also be reported by the camera?,SOS-based speed,"What is another speed, besides the noise-based and saturation-based speeds, that can also be reported by the camera according to the film speed standards?"
Sumer,"The most impressive and famous of Sumerian buildings are the ziggurats, large layered platforms which supported temples. Sumerian cylinder seals also depict houses built from reeds not unlike those built by the Marsh Arabs of Southern Iraq until as recently as 400 CE. The Sumerians also developed the arch, which enabled them to develop a strong type of dome. They built this by constructing and linking several arches. Sumerian temples and palaces made use of more advanced materials and techniques,[citation needed] such as buttresses, recesses, half columns, and clay nails.",Where were the use of advanced materials and techniques on display in Sumer?,Sumerian temples and palaces,"Where were the advanced materials and techniques, such as buttresses and arches, used in the construction of temples and palaces in Sumer?"
"Ann_Arbor,_Michigan","Ann Arbor has a council-manager form of government. The City Council has 11 voting members: the mayor and 10 city council members. The mayor and city council members serve two-year terms: the mayor is elected every even-numbered year, while half of the city council members are up for election annually (five in even-numbered and five in odd-numbered years). Two council members are elected from each of the city's five wards. The mayor is elected citywide. The mayor is the presiding officer of the City Council and has the power to appoint all Council committee members as well as board and commission members, with the approval of the City Council. The current mayor of Ann Arbor is Christopher Taylor, a Democrat who was elected as mayor in 2014. Day-to-day city operations are managed by a city administrator chosen by the city council.",Who is elected every even numbered year?,mayor,Who is elected as mayor in Ann Arbor every even-numbered year?
John_von_Neumann,"Shortly before his death, when he was already quite ill, von Neumann headed the United States government's top secret ICBM committee, and it would sometimes meet in his home. It

In [12]:
SAMPLES_DIR = "samples"
os.makedirs(SAMPLES_DIR, exist_ok=True)
dfSample.to_pickle(os.path.join(SAMPLES_DIR, f"samples.pkl"))    

In [8]:
prompt = "What year was the Banská Akadémia, the world's first institution of technology, founded in Banská Štiavnica, Slovakia?"
agent = get_agent(use_openai=True)
agent.run(prompt, stream=False, reset=True)


======== New task ========
What year was the Banská Akadémia, the world's first institution of technology, founded in Banská Štiavnica, Slovakia?
=== Agent thoughts:
Thought: I will use the squad_retriever tool to find information about the Banská Akadémia, specifically focusing on its founding year. I will include details about its significance as the world's first institution of technology and its location in Banská Štiavnica, Slovakia to refine my query.
>>> Agent is executing the code below:
answer = squad_retriever(query="What year was the Banská Akadémia founded in Banská Štiavnica, Slovakia?")
print(answer)
====
Print outputs:
===Document===
Title: Institute_of_technology
Context: The world's first institution of technology or technical university with tertiary technical education is the Banská Akadémia in Banská Štiavnica, Slovakia, founded in 1735, Academy since December 13, 1762 established by queen Maria Theresa in order to train specialists of silver and gold mining and met

"The Banská Akadémia, the world's first institution of technology, was founded in 1735."

### Create the agent to be evaluated

### Run the agent on the random sample of questions

* Unlike the default Retrieval QA or Open Generative QA of SQuAD, in our use case, the agent would normally be given context in the course of a natural conversation, as the user elaborates on what they want to know. 
* Therefore, for benchmarking, we will provide the context to answer the question in the prompt.

### Use semantic similarity to evaluate the agent's answers against the reference answers

* One flaw of this approach is that it does not take into account the existence of multiple acceptable answers.
* Another flaw is that the agent me be unfairly penalized for elaborating on the answer, while this benchmark focuses on only and exactly the one canonical answer given.


In [9]:
BENCHMARKS_DIR = "benchmarks"

def benchmark_agent(agent, dfSample, name):
    answers_ref, answers_pred = [], []        

    # Suppress logging from the agent, which can be quite verbose
    agent.logger.setLevel(logging.CRITICAL)

    for title, context, question, answer, synthesized_question in tqdm(dfSample.values):
        class Output:
            output: agent_types.AgentType | str = None

        prompt = synthesized_question
        answers_ref.append(answer)
        final_answer = agent.run(prompt, stream=False, reset=True)
        answers_pred.append(final_answer)

    answers_ref = [str(answer) for answer in answers_ref]
    answers_pred = [str(answer) for answer in answers_pred]

    em = EmbeddingModelWrapper()
    similarities = em.get_similarities(
        em.get_embeddings( answers_pred ),
        em.get_embeddings( answers_ref ),
    )

    dfAnswers = dfSample.copy()
    dfAnswers["Predicted Answer"] = answers_pred
    dfAnswers["Similarity"] = similarities

    os.makedirs(BENCHMARKS_DIR, exist_ok=True)
    dfAnswers.to_pickle(os.path.join(BENCHMARKS_DIR, f"{name}.pkl"))


### Set up and run the benchmarks

In [10]:
benchmarks = [
    (get_agent(), "baseline"), # Baseline agent with default settings
    (get_agent(system_prompt=SUCCINCT_SQUAD_REACT_CODE_SYSTEM_PROMPT), "succinct"), # Succinct agent
    (get_agent(system_prompt=FOCUSED_SQUAD_REACT_CODE_SYSTEM_PROMPT), "focused"), # Focused agent
]

for agent, name in tqdm(benchmarks):
    benchmark_agent(agent, dfSample, name)

  0%|          | 0/3 [00:00<?, ?it/s]

  0%|          | 0/10 [00:00<?, ?it/s]

  0%|          | 0/10 [00:00<?, ?it/s]

  0%|          | 0/10 [00:00<?, ?it/s]

In [23]:
# Load and display all benchmarks
def load_benchmarks():
    benchmarks_dir = "benchmarks"
    benchmarks = []
    for file in os.listdir(benchmarks_dir):
        if file.endswith(".pkl"):
            df = pd.read_pickle(os.path.join(benchmarks_dir, file))
            benchmarks.append({
                'name': file.replace(".pkl", ""),
                'data': df, 
                'mean_similarity': df['Similarity'].mean()
            })
    return benchmarks

benchmarks = load_benchmarks()

# Sort benchmarks by mean similarity
benchmarks.sort(key=lambda x: x['mean_similarity'], reverse=True)

for benchmark in benchmarks:
    display(Markdown(f"## {benchmark['name']}"))
    display(Markdown(f"### Mean similarity: {round(benchmark['mean_similarity'], 2)}"))
    display_text_df(benchmark['data'])



## focused

### Mean similarity: 0.89

Title,Context,Question,Answer,Synthesized Question,Predicted Answer,Similarity
Institute_of_technology,"The world's first institution of technology or technical university with tertiary technical education is the Banská Akadémia in Banská Štiavnica, Slovakia, founded in 1735, Academy since December 13, 1762 established by queen Maria Theresa in order to train specialists of silver and gold mining and metallurgy in neighbourhood. Teaching started in 1764. Later the department of Mathematics, Mechanics and Hydraulics and department of Forestry were settled. University buildings are still at their place today and are used for teaching. University has launched the first book of electrotechnics in the world.",What year was the Banská Akadémia founded?,1735,"What year was the Banská Akadémia, the world's first institution of technology, founded in Banská Štiavnica, Slovakia?",1735,1.000000
Film_speed,"The standard specifies how speed ratings should be reported by the camera. If the noise-based speed (40:1) is higher than the saturation-based speed, the noise-based speed should be reported, rounded downwards to a standard value (e.g. 200, 250, 320, or 400). The rationale is that exposure according to the lower saturation-based speed would not result in a visibly better image. In addition, an exposure latitude can be specified, ranging from the saturation-based speed to the 10:1 noise-based speed. If the noise-based speed (40:1) is lower than the saturation-based speed, or undefined because of high noise, the saturation-based speed is specified, rounded upwards to a standard value, because using the noise-based speed would lead to overexposed images. The camera may also report the SOS-based speed (explicitly as being an SOS speed), rounded to the nearest standard speed rating.",What is another speed that can also be reported by the camera?,SOS-based speed,"What is another speed, besides the noise-based and saturation-based speeds, that can also be reported by the camera according to the film speed standards?",SOS-based speed,1.000000
Sumer,"The most impressive and famous of Sumerian buildings are the ziggurats, large layered platforms which supported temples. Sumerian cylinder seals also depict houses built from reeds not unlike those built by the Marsh Arabs of Southern Iraq until as recently as 400 CE. The Sumerians also developed the arch, which enabled them to develop a strong type of dome. They built this by constructing and linking several arches. Sumerian temples and palaces made use of more advanced materials and techniques,[citation needed] such as buttresses, recesses, half columns, and clay nails.",Where were the use of advanced materials and techniques on display in Sumer?,Sumerian temples and palaces,"Where were the advanced materials and techniques, such as buttresses and arches, used in the construction of temples and palaces in Sumer?",Sumerian temples and palaces.,0.958309
"Ann_Arbor,_Michigan","Ann Arbor has a council-manager form of government. The City Council has 11 voting members: the mayor and 10 city council members. The mayor and city council members serve two-year terms: the mayor is elected every even-numbered year, while half of the city council members are up for election annually (five in even-numbered and five in odd-numbered years). Two council members are elected from each of the city's five wards. The mayor is elected citywide. The mayor is the presiding officer of the City Council and has the power to appoint all Council committee members as well as board and commission members, with the approval of the City Council. The current mayor of Ann Arbor is Christopher Taylor, a Democrat who was elected as mayor in 2014. Day-to-day city operations are managed by a city administrator chosen by the city council.",Who is elected every even numbered year?,mayor,Who is elected as mayor in Ann Arbor every even-numbered year?,Christopher Taylor,0.288215
John_von_Neumann,"Shortly before his death, when he was alrea

## succinct

### Mean similarity: 0.79

Title,Context,Question,Answer,Synthesized Question,Predicted Answer,Similarity
Institute_of_technology,"The world's first institution of technology or technical university with tertiary technical education is the Banská Akadémia in Banská Štiavnica, Slovakia, founded in 1735, Academy since December 13, 1762 established by queen Maria Theresa in order to train specialists of silver and gold mining and metallurgy in neighbourhood. Teaching started in 1764. Later the department of Mathematics, Mechanics and Hydraulics and department of Forestry were settled. University buildings are still at their place today and are used for teaching. University has launched the first book of electrotechnics in the world.",What year was the Banská Akadémia founded?,1735,"What year was the Banská Akadémia, the world's first institution of technology, founded in Banská Štiavnica, Slovakia?",1735,1.000000
Film_speed,"The standard specifies how speed ratings should be reported by the camera. If the noise-based speed (40:1) is higher than the saturation-based speed, the noise-based speed should be reported, rounded downwards to a standard value (e.g. 200, 250, 320, or 400). The rationale is that exposure according to the lower saturation-based speed would not result in a visibly better image. In addition, an exposure latitude can be specified, ranging from the saturation-based speed to the 10:1 noise-based speed. If the noise-based speed (40:1) is lower than the saturation-based speed, or undefined because of high noise, the saturation-based speed is specified, rounded upwards to a standard value, because using the noise-based speed would lead to overexposed images. The camera may also report the SOS-based speed (explicitly as being an SOS speed), rounded to the nearest standard speed rating.",What is another speed that can also be reported by the camera?,SOS-based speed,"What is another speed, besides the noise-based and saturation-based speeds, that can also be reported by the camera according to the film speed standards?",SOS-based speed,1.000000
Sumer,"The most impressive and famous of Sumerian buildings are the ziggurats, large layered platforms which supported temples. Sumerian cylinder seals also depict houses built from reeds not unlike those built by the Marsh Arabs of Southern Iraq until as recently as 400 CE. The Sumerians also developed the arch, which enabled them to develop a strong type of dome. They built this by constructing and linking several arches. Sumerian temples and palaces made use of more advanced materials and techniques,[citation needed] such as buttresses, recesses, half columns, and clay nails.",Where were the use of advanced materials and techniques on display in Sumer?,Sumerian temples and palaces,"Where were the advanced materials and techniques, such as buttresses and arches, used in the construction of temples and palaces in Sumer?",Sumerian temples and palaces,1.000000
"Ann_Arbor,_Michigan","Ann Arbor has a council-manager form of government. The City Council has 11 voting members: the mayor and 10 city council members. The mayor and city council members serve two-year terms: the mayor is elected every even-numbered year, while half of the city council members are up for election annually (five in even-numbered and five in odd-numbered years). Two council members are elected from each of the city's five wards. The mayor is elected citywide. The mayor is the presiding officer of the City Council and has the power to appoint all Council committee members as well as board and commission members, with the approval of the City Council. The current mayor of Ann Arbor is Christopher Taylor, a Democrat who was elected as mayor in 2014. Day-to-day city operations are managed by a city administrator chosen by the city council.",Who is elected every even numbered year?,mayor,Who is elected as mayor in Ann Arbor every even-numbered year?,Christopher Taylor,0.288214
John_von_Neumann,"Shortly before his death, when he was alread

## baseline

### Mean similarity: 0.47

Title,Context,Question,Answer,Synthesized Question,Predicted Answer,Similarity
Institute_of_technology,"The world's first institution of technology or technical university with tertiary technical education is the Banská Akadémia in Banská Štiavnica, Slovakia, founded in 1735, Academy since December 13, 1762 established by queen Maria Theresa in order to train specialists of silver and gold mining and metallurgy in neighbourhood. Teaching started in 1764. Later the department of Mathematics, Mechanics and Hydraulics and department of Forestry were settled. University buildings are still at their place today and are used for teaching. University has launched the first book of electrotechnics in the world.",What year was the Banská Akadémia founded?,1735,"What year was the Banská Akadémia, the world's first institution of technology, founded in Banská Štiavnica, Slovakia?","The Banská Akadémia, the world's first institution of technology, was founded in 1735 in Banská Štiavnica, Slovakia.",0.211984
Film_speed,"The standard specifies how speed ratings should be reported by the camera. If the noise-based speed (40:1) is higher than the saturation-based speed, the noise-based speed should be reported, rounded downwards to a standard value (e.g. 200, 250, 320, or 400). The rationale is that exposure according to the lower saturation-based speed would not result in a visibly better image. In addition, an exposure latitude can be specified, ranging from the saturation-based speed to the 10:1 noise-based speed. If the noise-based speed (40:1) is lower than the saturation-based speed, or undefined because of high noise, the saturation-based speed is specified, rounded upwards to a standard value, because using the noise-based speed would lead to overexposed images. The camera may also report the SOS-based speed (explicitly as being an SOS speed), rounded to the nearest standard speed rating.",What is another speed that can also be reported by the camera?,SOS-based speed,"What is another speed, besides the noise-based and saturation-based speeds, that can also be reported by the camera according to the film speed standards?",Another speed that can be reported by the camera according to film speed standards is the SOS-based speed.,0.668879
Sumer,"The most impressive and famous of Sumerian buildings are the ziggurats, large layered platforms which supported temples. Sumerian cylinder seals also depict houses built from reeds not unlike those built by the Marsh Arabs of Southern Iraq until as recently as 400 CE. The Sumerians also developed the arch, which enabled them to develop a strong type of dome. They built this by constructing and linking several arches. Sumerian temples and palaces made use of more advanced materials and techniques,[citation needed] such as buttresses, recesses, half columns, and clay nails.",Where were the use of advanced materials and techniques on display in Sumer?,Sumerian temples and palaces,"Where were the advanced materials and techniques, such as buttresses and arches, used in the construction of temples and palaces in Sumer?","In Sumer, advanced materials and techniques such as buttresses and arches were used in the construction of temples and palaces.",0.720199
"Ann_Arbor,_Michigan","Ann Arbor has a council-manager form of government. The City Council has 11 voting members: the mayor and 10 city council members. The mayor and city council members serve two-year terms: the mayor is elected every even-numbered year, while half of the city council members are up for election annually (five in even-numbered and five in odd-numbered years). Two council members are elected from each of the city's five wards. The mayor is elected citywide. The mayor is the presiding officer of the City Council and has the power to appoint all Council committee members as well as board and commission members, with the approval of the City Council. The current mayor of Ann Arbor is Christopher Taylor, a Democrat who was elected as m